In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -nc -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!pip install -q findspark
!pip install pyspark

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 14.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=801e6e2a66ac18672903fee154f52c3566bee447d9cfdbe482743b85c6aafee5
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [3]:
spark = SparkSession \
       .builder \
       .appName("iot") \
       .getOrCreate()

spark

bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by bash)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/15 18:30:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Read the data into a Dataframe**

In [4]:
import os
for dirname, _, filenames in os.walk('../input/iot-devices'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/iot-devices/iot_devices.json


In [5]:
df = spark.read.json("../input/iot-devices/iot_devices.json")

In [6]:
df.printSchema()

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [7]:
df.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

**Convert the Dataframe into a temporary view called iot**

In [8]:
df.createOrReplaceTempView("iot")

In [9]:
df1 = spark.sql("SELECT * FROM iot")
df1.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

**Count how many devices are there from each country and display the output**

In [10]:
spark.sql("SELECT cn, COUNT (device_name) AS no_of_device FROM iot GROUP BY cn ORDER BY cn").show(50)

+--------------------+------------+
|                  cn|no_of_device|
+--------------------+------------+
|                    |        1810|
|         Afghanistan|          19|
|             Albania|          32|
|             Algeria|          34|
|      American Samoa|           3|
|             Andorra|           4|
|              Angola|          28|
|            Anguilla|           7|
| Antigua and Barbuda|          60|
|           Argentina|         978|
|             Armenia|          34|
|               Aruba|           8|
|           Australia|        3119|
|             Austria|        1462|
|          Azerbaijan|          49|
|             Bahamas|          18|
|             Bahrain|          55|
|          Bangladesh|         153|
|            Barbados|          38|
|             Belarus|         123|
|             Belgium|         666|
|              Belize|          13|
|               Benin|           3|
|             Bermuda|          53|
|              Bhutan|      

In [11]:
spark.sql("select cca3, count(distinct device_id) as device_id from iot group by cca3 order by device_id desc limit 100").show()

+----+---------+
|cca3|device_id|
+----+---------+
| USA|    70405|
| CHN|    14455|
| JPN|    12100|
| KOR|    11879|
| DEU|     7942|
| GBR|     6486|
| CAN|     6041|
| RUS|     5989|
| FRA|     5305|
| BRA|     3224|
| AUS|     3119|
| ITA|     2915|
| SWE|     2880|
| POL|     2744|
| NLD|     2488|
| ESP|     2310|
| TWN|     2128|
| IND|     1867|
| CZE|     1507|
| NOR|     1487|
+----+---------+
only showing top 20 rows



**Display all the countries whose carbon dioxide level is more than 1400. Sort the output in descending order.**

In [12]:
spark.sql("select cca3, c02_level from iot where c02_level > 1400 order by c02_level desc").show()

+----+---------+
|cca3|c02_level|
+----+---------+
| BRA|     1599|
| USA|     1599|
| USA|     1599|
| USA|     1599|
| USA|     1599|
| USA|     1599|
| CZE|     1599|
| USA|     1599|
| USA|     1599|
| UKR|     1599|
| RUS|     1599|
| IRL|     1599|
| CHN|     1599|
| USA|     1599|
| USA|     1599|
| RUS|     1599|
| UKR|     1599|
| CHN|     1599|
| KOR|     1599|
| USA|     1599|
+----+---------+
only showing top 20 rows



In [13]:
spark.sql("SELECT cn,c02_level FROM iot WHERE c02_level > 1400 ORDER BY c02_level DESC").show()

+-----------------+---------+
|               cn|c02_level|
+-----------------+---------+
|           Brazil|     1599|
|    United States|     1599|
|    United States|     1599|
|    United States|     1599|
|    United States|     1599|
|    United States|     1599|
|   Czech Republic|     1599|
|    United States|     1599|
|    United States|     1599|
|          Ukraine|     1599|
|           Russia|     1599|
|          Ireland|     1599|
|            China|     1599|
|    United States|     1599|
|    United States|     1599|
|           Russia|     1599|
|          Ukraine|     1599|
|            China|     1599|
|Republic of Korea|     1599|
|    United States|     1599|
+-----------------+---------+
only showing top 20 rows



**Select all countries' devices with high-levels of C02 and group by cca3 and order by device_ids (Hint: For high CO2 level, the LCD status will be RED).**

In [14]:
spark.sql("select cca3, count(distinct device_id) as device_id from iot where lcd == 'red' group by cca3 order by device_id").show()

+----+---------+
|cca3|device_id|
+----+---------+
| TJK|        1|
| REU|        1|
| TKM|        1|
| GIB|        1|
| CIV|        1|
| LBR|        1|
| MHL|        1|
| SLB|        1|
| GAB|        1|
| TLS|        1|
| IOT|        1|
| GGY|        1|
| FSM|        1|
| AND|        1|
| FLK|        1|
| VUT|        2|
| ZMB|        2|
| LCA|        2|
| GMB|        2|
| VCT|        2|
+----+---------+
only showing top 20 rows



In [15]:
spark.sql("SELECT cn,device_name,c02_level,lcd FROM iot WHERE lcd == 'red' ").show()

+-----------------+--------------------+---------+---+
|               cn|         device_name|c02_level|lcd|
+-----------------+--------------------+---------+---+
|           Norway|   sensor-pad-2n2Pea|     1473|red|
|            Italy| device-mac-36TWSKiT|     1556|red|
|            Japan|sensor-pad-8xUD6p...|     1536|red|
|    United States|sensor-pad-10Bsyw...|     1470|red|
|            Italy|meter-gauge-11dlM...|     1544|red|
|    United States|sensor-pad-16aXmI...|     1425|red|
|    United States|meter-gauge-17zb8...|     1466|red|
|    United States|meter-gauge-19eg1...|     1531|red|
|            Japan|  sensor-pad-22oWV2D|     1522|red|
|           Canada|sensor-pad-24Pytz...|     1511|red|
|Republic of Korea|  device-mac-27P5wf2|     1597|red|
|Republic of Korea|sensor-pad-28Tsud...|     1502|red|
|          Ukraine|meter-gauge-47WsF9s8|     1454|red|
|           Sweden|meter-gauge-534fD...|     1446|red|
|    United States|sensor-pad-5410CW...|     1436|red|
|    Unite

**Find out all devices in countries whose batteries need replacements.**

In [16]:
spark.sql("select cca3, count(distinct device_id) as device_id from iot where battery_level == 0 group by cca3 order by device_id desc limit 100").show()

+----+---------+
|cca3|device_id|
+----+---------+
| USA|     7043|
| CHN|     1415|
| KOR|     1217|
| JPN|     1210|
| DEU|      760|
| GBR|      650|
| CAN|      612|
| RUS|      600|
| FRA|      582|
| BRA|      374|
| AUS|      322|
| SWE|      293|
| ITA|      287|
| POL|      278|
| NLD|      251|
| ESP|      223|
| TWN|      207|
| IND|      189|
| UKR|      149|
| HKG|      149|
+----+---------+
only showing top 20 rows



In [17]:
spark.sql("SELECT cn,device_name,battery_level FROM iot WHERE battery_level == 0").show()

+-----------------+--------------------+-------------+
|               cn|         device_name|battery_level|
+-----------------+--------------------+-------------+
|            Japan|sensor-pad-8xUD6p...|            0|
|    United States|sensor-pad-12Y2kIm0o|            0|
|    United States|meter-gauge-17zb8...|            0|
|          Germany| sensor-pad-448DeWGL|            0|
|           Canada|sensor-pad-80TY4d...|            0|
|Republic of Korea| sensor-pad-92vxuq7e|            0|
|    United States|sensor-pad-98mJQA...|            0|
|            Japan|meter-gauge-1075K...|            0|
|        Australia|device-mac-111WYt...|            0|
|    United States| sensor-pad-11663yUf|            0|
|            China|device-mac-117mcc...|            0|
|    United States|device-mac-123zvY...|            0|
|    United States|meter-gauge-131Lb...|            0|
|    United States|sensor-pad-1344UG...|            0|
|          Germany|sensor-pad-140tq1s6t|            0|
|         